In [19]:
# %%
import ee
import geemap
import geopandas as gpd
from pyproj import Geod
from pathlib import Path





In [ ]:
# Authenticate once (browser popup). After this, ee.Initialize() works.
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

print("✅ Earth Engine initialized")

In [30]:
# Export PNGs locally (recommended for README)
OUT_DIR = Path(r"C:\GIS\my_water_projects\openwater-shrinking-lake-monitor\outputs\images")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [12]:
# %%


AOI_GEOJSON = r"C:\GIS\my_water_projects\openwater-shrinking-lake-monitor\data\external\aoi.geojson"

gdf = gpd.read_file(AOI_GEOJSON)
print("CRS:", gdf.crs)




CRS: EPSG:4326


In [13]:
minx, miny, maxx, maxy = gdf.total_bounds  # uses your original gdf
aoi_bbox = ee.Geometry.Rectangle([minx, miny, maxx, maxy])

print("✅ Using bbox rectangle aoi_bbox")
print([minx, miny, maxx, maxy])

✅ Using bbox rectangle AOI
[np.float64(-107.21444572778415), np.float64(33.14571033916773), np.float64(-107.14274375499289), np.float64(33.31855197156376)]


#### take original AOI which is long vertically and short wide, change it to a square 

In [22]:
# %%

# Center of bbox
cx = (minx + maxx) / 2
cy = (miny + maxy) / 2

# Compute width/height in meters using geodesic distances
geod = Geod(ellps="WGS84")

# width: distance between left/right at center latitude
_, _, width_m = geod.inv(minx, cy, maxx, cy)

# height: distance between bottom/top at center longitude
_, _, height_m = geod.inv(cx, miny, cx, maxy)

print(f"width_m={width_m:,.0f}  height_m={height_m:,.0f}")

# Choose square side = max dimension * padding factor
padding = 1.40  # 20% larger than max dimension (tweak 1.1–1.5)
side_m = max(width_m, height_m) * padding

# Buffer radius = half the square side
buffer_m = side_m / 2
print(f"side_m={side_m:,.0f}  buffer_m={buffer_m:,.0f}")

# Create a "square-ish" rectangle AOI in EE:
# buffer() uses meters (geodesic), bounds() makes a lat/lon-aligned rectangle
aoi_square = ee.Geometry.Point([cx, cy]).buffer(buffer_m).bounds()

print("✅ Created aoi_square")


width_m=6,683  height_m=19,170
side_m=26,837  buffer_m=13,419
✅ Created aoi_square


In [23]:
# %%


Map = geemap.Map()
Map.centerObject(aoi_square, 11)

Map.addLayer(aoi_square, {}, "AOI square (export)")
Map.addLayer(ee.Geometry.Rectangle([minx, miny, maxx, maxy]), {}, "Original bbox")
Map


Map(center=[33.2321261930872, -107.17834499385486], controls=(WidgetControl(options=['position', 'transparent_…

In [24]:
# %%
def mask_s2_sr_clouds(img: ee.Image) -> ee.Image:
    """
    Basic cloud mask for Sentinel-2 Surface Reflectance (S2_SR_HARMONIZED) using QA60 bits.
    Bit 10 = clouds, Bit 11 = cirrus.

    Returns a reflectance-scaled image in 0–1 (divide by 10000).
    """
    qa = img.select("QA60")
    cloud_bit = 1 << 10
    cirrus_bit = 1 << 11

    mask = qa.bitwiseAnd(cloud_bit).eq(0).And(qa.bitwiseAnd(cirrus_bit).eq(0))
    return img.updateMask(mask).divide(10000)


def rgb_monthly_composite(year: int, month: int, aoi: ee.Geometry) -> ee.Image:
    """
    Build a median RGB composite for a given (year, month) over the AOI.
    Uses lenient CLOUDY_PIXEL_PERCENTAGE filtering, then a QA60 mask.
    """
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, "month")

    col = (
        ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
        .filterBounds(aoi)
        .filterDate(start, end)
        .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 80))  # broad filter
        .map(mask_s2_sr_clouds)
    )

    # Median composite reduces residual cloud noise and fills coverage gaps across the month
    comp = col.median().clip(aoi)

    return comp


In [36]:
# %%
# Quick visual sanity check in an interactive map
Map = geemap.Map()
Map.centerObject(aoi_square, 11)

img_2019 = rgb_monthly_composite(2019, 9, aoi_square)
img_2025 = rgb_monthly_composite(2025, 9, aoi_square)

vis = {"min": 0.05, "max": 0.40, "bands": ["B4", "B3", "B2"], "gamma": 1.3}

Map.addLayer(img_2019, vis, "2019-09 RGB (median)")
Map.addLayer(img_2025, vis, "2025-09 RGB (median)")

aoi_outline = ee.FeatureCollection([ee.Feature(aoi_square)])
Map.addLayer(
    aoi_outline.style(**{"color": "yellow", "width": 2, "fillColor": "00000000"}),
    {},
    "AOI Square (outline)"
)

Map 


Map(center=[33.2321261930872, -107.17834499385486], controls=(WidgetControl(options=['position', 'transparent_…

In [61]:


def export_rgb_to_drive_vis_native(img: ee.Image, aoi: ee.Geometry, name: str, folder: str = "openwater_rgb", scale: int = 10):
    task = ee.batch.Export.image.toDrive(
        image=img.visualize(**vis),   # matches your Map.addLayer vis
        description=name,
        folder=folder,
        fileNamePrefix=name,
        region=aoi,
        scale=scale,
        maxPixels=1e13
    )
    task.start()
    return task

task_2019_v3 = export_rgb_to_drive_vis_native(img_2019, aoi_square, "s2_rgb_2019_09_v3")
task_2025_v3 = export_rgb_to_drive_vis_native(img_2025, aoi_square, "s2_rgb_2025_09_v3")

print("✅ Started native Drive exports:", task_2019_v3.id, task_2025_v3.id)


✅ Started native Drive exports: C3FBHIAZTDUHA6ZWG7QMTQUL RA6EAHMGA57KHBJ7H4O342P2


In [62]:
for t in [task_2019_v3, task_2025_v3]:
    print(t.id, t.status().get("state"), t.status().get("description"))


C3FBHIAZTDUHA6ZWG7QMTQUL READY s2_rgb_2019_09_v3
RA6EAHMGA57KHBJ7H4O342P2 READY s2_rgb_2025_09_v3


In [60]:
targets = {"s2_rgb_2019_09", "s2_rgb_2025_09"}

tasks = ee.batch.Task.list()

for t in tasks:
    status = t.status()
    desc = status.get("description")
    state = status.get("state")
    if desc in targets and state in {"READY", "RUNNING"}:
        t.cancel()
        print("Canceled:", t.id, state, desc)


In [58]:
tasks = ee.batch.Task.list()
for t in tasks[:20]:
    s = t.status()
    if "s2_rgb_" in (s.get("description") or ""):
        print(t.id, s.get("state"), s.get("description"))


FVRE4NZP7FABGMAEWE6TQTRJ READY s2_rgb_2025_09_v2
G5M2QJM2NH52BDV23WE5F3F2 READY s2_rgb_2019_09_v2
Q3WPC5KBDTRXOSUQCVP4C5M3 CANCELLED s2_rgb_2025_09
2T7SDLF2RXTACKDVU4GV7O3W CANCELLED s2_rgb_2019_09
4NSV5BRHBYWZV3B3ZGD7VSWB CANCELLED s2_rgb_2025_09
7LBMFN3T4NBJWAY56XZS2CTH CANCEL_REQUESTED s2_rgb_2019_09
ZT64JZLAKWEKS23SPZJJZESK CANCEL_REQUESTED s2_rgb_2025_09
VYO2LM5WVSAW6BDPFLHPW242 COMPLETED s2_rgb_2019_09
RSNGPUGBMFDR4I4RGJAABXRF COMPLETED s2_rgb_2025_09
7XP4SVPPFDUO4KKJNILB52FY COMPLETED s2_rgb_2019_09
MRI642CDUFM3NIPY2HF645ID COMPLETED s2_rgb_2025_09
5SBSGJYAWAW2TUFSKZXLFF4B COMPLETED s2_rgb_2019_09
EEVUL5JNI35AODSIUQPQ2FOS COMPLETED s2_rgb_2025_09
CTMLNRPHJWQHPZR45T56YOAM COMPLETED s2_rgb_2019_09
